In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_growth_ratio")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   id              5759 non-null   int64              
 1   ticker          5759 non-null   object             
 2   stac_yymm       5759 non-null   object             
 3   grs             5759 non-null   object             
 4   bsop_prfi_inrt  5759 non-null   object             
 5   equt_inrt       5759 non-null   object             
 6   totl_aset_inrt  5759 non-null   object             
 7   updated_at      5759 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), object(6)
memory usage: 360.1+ KB


,id,ticker,stac_yymm,grs,bsop_prfi_inrt,equt_inrt,totl_aset_inrt,updated_at
0,1,005930,202506,5.2900,-33.3600,4.18,3.94,2025-09-16 13:05:51.951039+00:00
1,2,005930,202503,10.0500,1.2000,9.33,9.66,2025-09-16 13:05:51.951039+00:00
2,3,005930,202412,16.2000,398.3400,10.59,12.86,2025-09-16 13:05:51.951039+00:00
3,4,005930,202409,17.7500,601.0000,6.15,8.11,2025-09-16 13:05:51.951039+00:00
4,5,005930,202406,17.9700,1202.7900,6.84,8.43,2025-09-16 13:05:51.951039+00:00
...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,14.1600,80.4100,9.07,6.56,2025-09-16 13:05:51.951039+00:00
5755,5756,002710,201812,5.5700,110.7000,11.23,9.36,2025-09-16 13:05:51.951039+00:00
5756,5757,002710,201809,0.9200,72.6500,4.52,1.00,2025-09-16 13:05:51.951039+00:00
5757,5758,002710,201806,-1.3200,56.7500,2.42,-2.47,2025-09-16 13:05:51.951039+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
for col in final_cols:
  df[col] .replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df[col].replace(['0', 0], np.nan, inplace=True)

missing_value_counts = df[final_cols].isnull().sum()

missing_value_counts

ticker            0
stac_yymm         0
grs               0
bsop_prfi_inrt    0
equt_inrt         0
totl_aset_inrt    0
updated_at        0
dtype: int64

In [5]:
missing_cols = missing_value_counts[missing_value_counts >= 100].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 7] - [결측 컬럼 : 0] = [후보 컬럼 : 7]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.tolist()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['updated_at']
[원본 컬럼 : 7] - [결측 컬럼 : 0] - [유니크 컬럼 : 1] = [후보 컬럼 : 6]


In [7]:
df[final_cols]

,ticker,stac_yymm,grs,bsop_prfi_inrt,equt_inrt,totl_aset_inrt
0,005930,202506,5.2900,-33.3600,4.18,3.94
1,005930,202503,10.0500,1.2000,9.33,9.66
2,005930,202412,16.2000,398.3400,10.59,12.86
3,005930,202409,17.7500,601.0000,6.15,8.11
4,005930,202406,17.9700,1202.7900,6.84,8.43
...,...,...,...,...,...,...
5754,002710,201903,14.1600,80.4100,9.07,6.56
5755,002710,201812,5.5700,110.7000,11.23,9.36
5756,002710,201809,0.9200,72.6500,4.52,1.00
5757,002710,201806,-1.3200,56.7500,2.42,-2.47


In [8]:
print(final_cols)

['ticker', 'stac_yymm', 'grs', 'bsop_prfi_inrt', 'equt_inrt', 'totl_aset_inrt']


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | **필수** | 어떤 기업의 성장성 지표인지를 특정하는 핵심 식별자입니다. 모든 데이터를 연결하기 위해 반드시 필요합니다. |
| `stac_yymm` | 결산년월 | **필수** | 데이터가 어느 시점의 실적인지를 나타내는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 이벤트와 결합할 때 필수적입니다. |
| `grs` | 매출액증가율 | **상** | 기업의 외형적 성장세를 보여주는 가장 기본적인 지표입니다. 시장 기대치를 상회하거나 하회할 경우 주가 변동성에 직접적인 영향을 줍니다. |
| `bsop_prfi_inrt` | 영업이익증가율 | **상** | 기업의 핵심 영업활동으로 인한 이익의 성장성을 나타냅니다. 매출 성장보다 수익성의 질을 보여주므로 변동성 예측에 더 민감한 피처가 될 수 있습니다. |
| `equt_inrt` | 자기자본증가율 | **중** | 기업의 순자산이 얼마나 성장했는지를 보여주는 지표입니다. 장기적인 안정성과 성장성을 나타내지만, 단기적인 뉴스 기반 변동성에는 덜 민감할 수 있습니다. |
| `totl_aset_inrt` | 총자산증가율 | **중** | 기업의 전체 규모 성장률을 나타냅니다. 투자나 인수를 통해 자산이 급격히 변할 경우 중요한 신호가 될 수 있으나, 일반적으로는 이익 지표보다 후행적입니다. |

In [9]:
final_cols = [
  'ticker', 
  'stac_yymm', 
  'grs', 
  'bsop_prfi_inrt', 
  'equt_inrt', 
  'totl_aset_inrt'
]

In [10]:
df[final_cols]

,ticker,stac_yymm,grs,bsop_prfi_inrt,equt_inrt,totl_aset_inrt
0,005930,202506,5.2900,-33.3600,4.18,3.94
1,005930,202503,10.0500,1.2000,9.33,9.66
2,005930,202412,16.2000,398.3400,10.59,12.86
3,005930,202409,17.7500,601.0000,6.15,8.11
4,005930,202406,17.9700,1202.7900,6.84,8.43
...,...,...,...,...,...,...
5754,002710,201903,14.1600,80.4100,9.07,6.56
5755,002710,201812,5.5700,110.7000,11.23,9.36
5756,002710,201809,0.9200,72.6500,4.52,1.00
5757,002710,201806,-1.3200,56.7500,2.42,-2.47


In [11]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   grs             5759 non-null   object        
 3   bsop_prfi_inrt  5759 non-null   object        
 4   equt_inrt       5759 non-null   object        
 5   totl_aset_inrt  5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(4)
memory usage: 246.0+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 | 적정 타입 | 이유 및 해결 방안 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 종목 코드 | `category` | `category` | 메모리 효율성을 위해 `category`로 변환된 상태가 맞습니다. 추가 작업이 필요 없습니다. |
| `stac_yymm` | 결산년월 | `datetime64[ns]` | `datetime64[ns]` | 시계열 분석을 위해 `datetime`으로 변환된 상태가 맞습니다. 추가 작업이 필요 없습니다. |
| `grs` | 매출액증가율 | `object` | `float64` | 소수점과 음수를 포함하는 비율 데이터이므로 `float64`가 적합합니다. `pd.to_numeric`을 사용해 강제로 숫자 변환이 필요합니다. |
| `bsop_prfi_inrt` | 영업이익증가율 | `object` | `float64` | 다른 성장률 지표와 마찬가지로, 숫자 외 다른 문자가 포함되어 변환에 실패한 것으로 보입니다. `float64`로 변환해야 합니다. |
| `equt_inrt` | 자기자본증가율 | `object` | `float64` | 데이터의 일관성과 정확한 계산을 위해 다른 비율 데이터와 동일하게 `float64` 타입으로 관리해야 합니다. |
| `totl_aset_inrt` | 총자산증가율 | `object` | `float64` | 현재 `object`로 남아있는 모든 성장률 지표는 `pd.to_numeric`을 이용해 `float64` 타입으로 변환을 완료해야 합니다. |

In [13]:
dtype_map = {
  'float64': ['grs', 'bsop_prfi_inrt', 'equt_inrt', 'totl_aset_inrt']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   ticker          5759 non-null   category      
 1   stac_yymm       5759 non-null   datetime64[ns]
 2   grs             5759 non-null   float64       
 3   bsop_prfi_inrt  5759 non-null   float64       
 4   equt_inrt       5759 non-null   float64       
 5   totl_aset_inrt  5759 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(4)
memory usage: 246.0 KB
